## **Libraries and Dependencies**

In [1]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, optimizers, applications
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import RandomZoom, RandomRotation, RandomFlip, Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from PIL import Image
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import pandas as pd
import cv2
import logging
import warnings
import json


# Suppress warnings from the logging module
logging.getLogger('tensorflow').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning)


## **Tensorflow Version**

In [2]:
# Tensorflow Version
print(tf.__version__)

2.10.0


## **GPU Checker**

In [3]:
# Check if any GPU devices are detected
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs detected: {len(gpus)}")
else:
    print("No GPU detected.")

GPUs detected: 1


In [5]:
# Suppress TensorFlow logging except for fatal errors.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## **Global Variables**

In [6]:
# Set seed
SEED = 338424

# Global variables
IMG_SIZE = (64, 64)
BATCH_SIZE = 32
num_classes = 18 # Number of folders in dataset
AUTOTUNE = tf.data.AUTOTUNE

## **Dataset: Loading, Splitting, Shuffling, Caching**

In [44]:
# Load Dataset
dataset_dir = 'dataset/hagridset'
full_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    shuffle=True,
    seed=SEED,
    image_size=(IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Split into training, validation, and test sets
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Total length of the dataset
total_size = len(full_ds)

# Compute indices for the splits
train_size = int(total_size * train_ratio)
val_size = int(total_size * val_ratio)
test_size = total_size - (train_size + val_size)

# Split the dataset and shuffle
train_ds = full_ds.take(train_size).shuffle(train_size, seed=SEED)
val_ds = full_ds.skip(train_size).take(val_size).shuffle(val_size, seed=SEED)
test_ds = full_ds.skip(train_size + val_size).shuffle(test_size, seed=SEED)

# Cache the dataset in memory (or use a directory to store it on disk if necessary)
train_ds = full_ds.take(train_size).shuffle(train_size, seed=SEED).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = full_ds.skip(train_size).take(val_size).shuffle(val_size, seed=SEED).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = full_ds.skip(train_size + val_size).cache().prefetch(buffer_size=AUTOTUNE)

# Count samples in each subset
def count_samples(dataset):
    sample_count = sum(1 for _ in dataset.unbatch())
    return sample_count

# Output the number of samples for each dataset
print(f'Using {count_samples(train_ds)} samples in the Training set')
print(f'Using {count_samples(val_ds)} samples in the Validation set')
print(f'Using {count_samples(test_ds)} samples in the Test set')

Found 125912 files belonging to 18 classes.
Using 88128 samples in the Training set
Using 25184 samples in the Validation set
Using 12600 samples in the Test set


In [45]:
# Get class names
class_names = full_ds.class_names
class_names

['call',
 'dislike',
 'fist',
 'four',
 'like',
 'mute',
 'ok',
 'one',
 'palm',
 'peace',
 'peace_inverted',
 'rock',
 'stop',
 'stop_inverted',
 'three',
 'three2',
 'two_up',
 'two_up_inverted']

#### **Dataset: Save**

In [ ]:
path_to_save = 'savedDataset'

tf.data.experimental.save(train_ds, path_to_save + '/train')
tf.data.experimental.save(val_ds, path_to_save + '/val')
tf.data.experimental.save(test_ds, path_to_save + '/test')

## **Regularization Factors**

This code snippet defines the values for L1 and L2 regularization, which are both set to 0.01. It then creates an "Elastic Net Regularizer" that combines these L1 and L2 values to help prevent the model from overfitting by penalizing overly complex or large weight values in the model's learning process.

In [9]:
# Define L1 and L2 regularization factors
l1_factor = 0.01  # Example value
l2_factor = 0.01  # Example value

# Elastic Net Regularizer
elastic_net_regularizer = regularizers.l1_l2(l1=l1_factor, l2=l2_factor)

## **Callbacks: Learning Rate Scheduler and Early Stopping**

In [10]:
# Define a learning rate schedule
def lr_time_based_decay(epoch, lr):
    # This function adjusts the learning rate over each epoch based on the initial learning rate,
    # applying a decay factor that increases with the epoch number. It effectively reduces the 
    # learning rate over time, which can help in calibrating the model adjustments as it 
    # approaches a minimum in the loss surface.
    return lr * 1 / (1 + 0.01 * epoch)

# Define callbacks
callbacks = [
    # EarlyStopping prevents overfitting by stopping training when the validation loss 
    # has not improved for 3 consecutive epochs ('patience=3'). It also restores the 
    # weights of the model to those of the epoch with the best validation loss, ensuring 
    # the model retains the best learned features even if it starts to overfit afterward.
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    # LearningRateScheduler adjusts the learning rate according to the lr_time_based_decay function above.
    # It logs the new learning rate at the start of each epoch ('verbose=1'), helping to control
    # the step size of model updates, which can be crucial for reaching convergence efficiently.
    LearningRateScheduler(lr_time_based_decay, verbose=1)
]

## **Data Augmentation Layers**

In [11]:
# Data Augmentation
data_augmentation_layers = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
])

---

# **Convolutional Neural Networks (CNN)**

---

---

# **CNN's from Scratch: Shallow**

---

## **CNN Model: Shallow**

In [12]:
# Define the CNN Model from Scratch
def build_scratch_cnn_shallow():
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
    model.add(layers.Rescaling(1.0 / 255))  # Normalize pixel values
    
    model.add(layers.Conv2D(16, 3, padding='same', activation='relu')) 
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))
    
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))  
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))
    
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))  
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))
    
    model.add(layers.Flatten())

    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile model with an initial learning rate
    model.compile(
        # Adam optimizer is used with a specified initial learning rate of 0.001. The learning rate
        # controls how much the weights of the model are adjusted relative to the gradient of the loss 
        # function. A higher learning rate might converge quickly, but too high can cause the training 
        # to diverge. A lower learning rate ensures more reliable convergence but at the risk of slowing
        # down the training process. The chosen rate of 0.001 is a starting point that balances these factors.
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
        )
    return model

# Instantiate and summarize the lighter model
scratch_model_shallow = build_scratch_cnn_shallow()
scratch_model_shallow.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 16)       64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 32, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)       

### **CNN Model: Shallow - Training**

In [13]:
# Train CNN Model
history_shallow = scratch_model_shallow.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/10
2754/2754 [==============================] - 31s 10ms/step - loss: 2.4293 - accuracy: 0.2959 - val_loss: 1.6483 - val_accuracy: 0.4700 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0009900990569281696.
Epoch 2/10
2754/2754 [==============================] - 28s 10ms/step - loss: 1.4087 - accuracy: 0.5466 - val_loss: 1.1886 - val_accuracy: 0.6182 - lr: 9.9010e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0009706853341092082.
Epoch 3/10
2754/2754 [==============================] - 27s 10ms/step - loss: 1.1214 - accuracy: 0.6379 - val_loss: 1.0414 - val_accuracy: 0.6680 - lr: 9.7069e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0009424129424128428.
Epoch 4/10
2754/2754 [==============================] - 27s 10ms/step - loss: 0.9851 - accuracy: 0.6848 - val_loss: 1.1234 - val_accuracy: 0.6430 - lr: 9.4241e-04

Epoch 5: LearningRateScheduler 

#### **CNN Model: Shallow - Save**

In [15]:
# Save Model: .json
# Saves the Model Architecture
for key in history_shallow.history.keys():
    history_shallow.history[key] = [float(i) for i in history_shallow.history[key]]

# Write the JSON file
with open('json/cnn_model_shallow.json', 'w') as f:
    json.dump(history_shallow.history, f)


# Save Model: .h5
# Saves the Model Weights and Configurations
scratch_model_shallow.save('h5/scratch_model_shallow.h5')

## **CNN Model: Shallow Calibrated**

In [17]:
# Define the CNN Model from Scratch
def build_scratch_cnn_shallow_calibrated():
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
    model.add(layers.Rescaling(1.0 / 255))  # Normalize pixel values
    
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu')) 
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))
    
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu')) 
    model.add(layers.BatchNormalization()) 
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))
    
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu'))  
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))

    model.add(layers.Conv2D(256, 3, padding='same', activation='relu'))  
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))

    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Flatten())

    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile model with an initial learning rate
    model.compile(
        # Adam optimizer is used with a specified initial learning rate of 0.001. The learning rate
        # controls how much the weights of the model are adjusted relative to the gradient of the loss 
        # function. A higher learning rate might converge quickly, but too high can cause the training 
        # to diverge. A lower learning rate ensures more reliable convergence but at the risk of slowing
        # down the training process. The chosen rate of 0.001 is a starting point that balances these factors.
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
        )
    
    return model

# Instantiate and summarize the lighter model
scratch_model_shallow_calibrated = build_scratch_cnn_shallow_calibrated()
scratch_model_shallow_calibrated.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 conv2d_7 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 batch_normalization_7 (Batc  (None, 64, 64, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 32, 32, 64)       

### **CNN Model: Shallow Calibrated - Training**

In [18]:
# Train the model
history_shallow_calibrated = scratch_model_shallow_calibrated.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks  # Include both callbacks here
)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/10
2754/2754 [==============================] - 52s 18ms/step - loss: 1.7793 - accuracy: 0.4315 - val_loss: 1.0155 - val_accuracy: 0.6743 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0009900990569281696.
Epoch 2/10
2754/2754 [==============================] - 54s 20ms/step - loss: 0.8863 - accuracy: 0.7128 - val_loss: 0.6168 - val_accuracy: 0.8032 - lr: 9.9010e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0009706853341092082.
Epoch 3/10
2754/2754 [==============================] - 52s 19ms/step - loss: 0.6724 - accuracy: 0.7829 - val_loss: 0.5324 - val_accuracy: 0.8290 - lr: 9.7069e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0009424129424128428.
Epoch 4/10
2754/2754 [==============================] - 49s 18ms/step - loss: 0.5610 - accuracy: 0.8189 - val_loss: 0.4719 - val_accuracy: 0.8515 - lr: 9.4241e-04

Epoch 5: LearningRateScheduler 

#### **CNN Model: Shallow Calibrated - Save**

In [20]:
# Save Model: .json
# Saves the Model Architecture
for key in history_shallow_calibrated.history.keys():
    history_shallow_calibrated.history[key] = [float(i) for i in history_shallow_calibrated.history[key]]

# Write the JSON file
with open('json/cnn_model_shallow_calibrated.json', 'w') as f:
    json.dump(history_shallow_calibrated.history, f)


# Save Model: .h5
# Saves the Model Weights and Configurations
scratch_model_shallow_calibrated.save('h5/scratch_model_shallow_calibrated.h5')

## **CNN Model: Shallow Calibrated + Data Augmentation**

In [36]:
# Define the CNN Model from Scratch
def build_scratch_cnn_shallow_calibrated_DA():
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
    model.add(data_augmentation_layers)
    model.add(layers.Rescaling(1.0 / 255))  # Normalize pixel values
    
    # L1 and L2 regularization is applied to the weights of the layer, with a regularization strength of 0.001.
    # This regularization helps prevent overfitting by penalizing large weights, encouraging the model to learn smaller, more general weights.
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))
    
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization()) 
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))
    
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))

    model.add(layers.Conv2D(256, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))

    # Elastic Net Regularization is applied to the weights of this layer, with a regularization strength of 0.001.
    # This regularization helps prevent overfitting by penalizing large weights, encouraging the model to learn smaller, more general weights.
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=elastic_net_regularizer))
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Flatten())

    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile model with an initial learning rate
    model.compile(
        # Adam optimizer is used with a specified initial learning rate of 0.001. The learning rate
        # controls how much the weights of the model are adjusted relative to the gradient of the loss 
        # function. A higher learning rate might converge quickly, but too high can cause the training 
        # to diverge. A lower learning rate ensures more reliable convergence but at the risk of slowing
        # down the training process. The chosen rate of 0.001 is a starting point that balances these factors.
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
        )
    
    return model

# Instantiate and summarize the lighter model
scratch_model_shallow_calibrated_DA = build_scratch_cnn_shallow_calibrated_DA()
scratch_model_shallow_calibrated_DA.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 64, 64, 3)         0         
                                                                 
 rescaling_12 (Rescaling)    (None, 64, 64, 3)         0         
                                                                 
 conv2d_45 (Conv2D)          (None, 64, 64, 32)        896       
                                                                 
 batch_normalization_44 (Bat  (None, 64, 64, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_44 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 dropout_47 (Dropout)        (None, 32, 32, 32)      

### **CNN Model: Shallow Calibrated Data Augmentation - Training**

In [37]:
# Train the model
history_shallow_calibrated_DA = scratch_model_shallow_calibrated_DA.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks  # Include both callbacks here
)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/10
2754/2754 [==============================] - 358s 129ms/step - loss: 3.3454 - accuracy: 0.3425 - val_loss: 1.7126 - val_accuracy: 0.5860 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0009900990569281696.
Epoch 2/10
2754/2754 [==============================] - 353s 128ms/step - loss: 1.5860 - accuracy: 0.6044 - val_loss: 1.2595 - val_accuracy: 0.7255 - lr: 9.9010e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0009706853341092082.
Epoch 3/10
2754/2754 [==============================] - 347s 126ms/step - loss: 1.3381 - accuracy: 0.6820 - val_loss: 1.3125 - val_accuracy: 0.7023 - lr: 9.7069e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0009424129424128428.
Epoch 4/10
2754/2754 [==============================] - 347s 126ms/step - loss: 1.2133 - accuracy: 0.7194 - val_loss: 1.1498 - val_accuracy: 0.7619 - lr: 9.4241e-04

Epoch 5: LearningRateSc

#### **CNN Model: Shallow Calibrated Data Augmentation - Save**

In [39]:
# Save Model: .json
# Saves the Model Architecture
for key in history_shallow_calibrated_DA.history.keys():
    history_shallow_calibrated_DA.history[key] = [float(i) for i in history_shallow_calibrated_DA.history[key]]

# Write the JSON file
with open('json/cnn_model_shallow_calibrated_DA.json', 'w') as f:
    json.dump(history_shallow_calibrated_DA.history, f)


# Save Model: .h5
# Saves the Model Weights and Configurations
scratch_model_shallow_calibrated_DA.save('h5/scratch_model_shallow_calibrated_DA.h5')

## **CNN Model: K-Fold Cross Validation**

In [41]:
# Define the Light CNN Model from Scratch
def build_scratch_cnn_shallow_kfold():
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
    model.add(layers.Rescaling(1.0 / 255))  # Normalize pixel values
    
    model.add(layers.Conv2D(16, 3, padding='same', activation='relu')) 
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))
    
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))  
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))
    
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))  
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))

    model.add(layers.Conv2D(128, 3, padding='same', activation='relu', kernel_regularizer=elastic_net_regularizer))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))

    # Elastic Net Regularization is applied to the weights of this layer, with a regularization strength of 0.001.
    # This regularization helps prevent overfitting by penalizing large weights, encouraging the model to learn smaller, more general weights.
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=elastic_net_regularizer))
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Flatten())

    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile model with an initial learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
        )
    return model

# Instantiate and summarize the lighter model
scratch_model_shallow_kfold = build_scratch_cnn_shallow_kfold()
scratch_model_shallow_kfold.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_13 (Rescaling)    (None, 64, 64, 3)         0         
                                                                 
 conv2d_49 (Conv2D)          (None, 64, 64, 16)        448       
                                                                 
 batch_normalization_48 (Bat  (None, 64, 64, 16)       64        
 chNormalization)                                                
                                                                 
 max_pooling2d_48 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 dropout_52 (Dropout)        (None, 32, 32, 16)        0         
                                                                 
 conv2d_50 (Conv2D)          (None, 32, 32, 32)      

### **CNN Model: K-Fold Cross Validation - Training**

In [43]:
# Function to implement k-fold cross-validation within the TensorFlow pipeline
def perform_k_fold(num_folds=5):
    # Prepare to collect fold results
    all_fold_history = []

    # Shuffle the dataset
    full_ds = full_ds.shuffle(buffer_size=len(full_ds))

    total_batches = int(tf.data.experimental.cardinality(full_ds).numpy())
    fold_size = total_batches // num_folds

    for fold in range(num_folds):
        print(f"Training on fold {fold+1}/{num_folds}...")

        val_start = fold * fold_size
        val_end = val_start + fold_size

        val_ds_fold = full_ds.skip(val_start).take(fold_size)
        train_ds_fold = full_ds.take(val_start).concatenate(full_ds.skip(val_end))

        scratch_model_shallow_kfold = build_scratch_cnn_shallow_kfold()

        # Train the model
        history_kfold = scratch_model_shallow_kfold.fit(
            train_ds_fold, 
            validation_data=val_ds_fold,
            epochs=10,
            callbacks=callbacks
        )
        all_fold_history.append(history_kfold.history)

    return all_fold_history

# Call the k-fold cross-validation function
perform_k_fold(num_folds=5)

UnboundLocalError: local variable 'full_ds' referenced before assignment

In [ ]:
# Call the k-fold function and store histories
fold_histories = perform_k_fold(num_folds=5)

# Example of plotting the average accuracy per epoch across all folds
def plot_k_fold_accuracy(histories):
    avg_accuracy = np.mean([hist['accuracy'] for hist in histories], axis=0)
    plt.figure(figsize=(8, 5))
    plt.plot(avg_accuracy, label='Average Training Accuracy')
    plt.title('Training Accuracy per Epoch Across All Folds')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Plotting the results
plot_k_fold_accuracy(fold_histories)

## **CNN Model: Shallow Greyscale**

---

# **CNN's from Scratch: Deep**

---

## **CNN Model: Deep**

## **CNN Model: Deep Calibrated**

## **CNN Model: Deep Calibrated + Data Augmentation**

## **CNN Model: Deep Greyscale**